# Assertion RLA

## Overview of the assertion audit tool

The tool requires as input:

+ audit-specific and contest-specific parameters, such as
    - whether to sample with or without replacement
    - the name of the risk function to use, and any parameters it requires
    - a risk limit for each contest to be audited
    - the social choice function for each contest, including the number of winners
    - candidate identifiers
+ a ballot manifest**
+ a random seed
+ a file of cast vote records
+ reported results for each contest
+ json files of assertions for IRV contests (one file per IRV contest)
+ human reading of voter intent from the paper ballots selected for audit

** The ballot manifest could be only for ballots purported to contain the
contests under audit (manifest_type == "STYLE"), or could include ballots that might not contain the
contest (manifest_type == "ALL"). These are treated differently. If the manifest lists only ballots purported
to contain the contest under audit and a sampled ballot turns out not to
contain the contest, that is considered a discrepancy, dealt with using the "phantoms to zombies" approach.

The tool helps select ballots for audit, and reports when the audit has found sufficiently strong evidence to stop.

The tool exports a log of all the audit inputs except the CVR file, but including the auditors' manually determined voter intent from the audited ballots.

The current version uses a single sample to audit all contests. It would be possible to refine things to target smaller contests.

In [1]:
from __future__ import division, print_function

import math
import json
import warnings
import numpy as np
import pandas as pd
import csv

from collections import OrderedDict
from IPython.display import display, HTML

from cryptorandom.cryptorandom import SHA256
from cryptorandom.sample import sample_by_index

from assertion_audit_utils import \
    Assertion, Assorter, CVR, TestNonnegMean, check_audit_parameters, write_audit_parameters
from dominion_tools import \
    prep_dominion_manifest, sample_from_manifest, write_ballots_sampled


# Audit parameters.

* `seed`: the numeric seed for the pseudo-random number generator used to draw sample 
* `replacement`: whether to sample with replacement. If the sample is drawn with replacement, gamma must also be specified.
* `risk_function`: the function to be used to measure risk. Options are `kaplan_markov`,`kaplan_wald`,`kaplan_kolmogorov`,`wald_sprt`,`kaplan_martingale`. Not all risk functions work with every social choice function. `wald_sprt` applies only to plurality contests.
* `g`: a parameter to hedge against the possibility of observing a maximum overstatement. Require $g \in [0, 1)$ for `kaplan_markov` and `kaplan_wald`
* `N_cards`: an upper bound on the number of pieces of paper cast in the contest. This should be derived independently of the voting system. A ballot consists of one or more cards.

----

* `cvr_file`: filename for CVRs (input)
* `manifest_file`: filename for ballot manifest (input)
* `manifest_type`: "STYLE" if the manifest is supposed to list only ballots that contain the contests under audit; "ALL" if the manifest contains all ballots cast in the election
* `assertion_file`: filename of assertions for IRV contests, in RAIRE format
* `sample_file`: filename for sampled ballot identifiers (output)
* `mvr_file`: filename for manually ascertained votes from sampled ballots (input)
* `log_file`: filename for audit log (output)

----

* `error_rates`: dict of expected error rates. The keys are
    + `o1_rate`: expected rate of 1-vote overstatements. Recommended value $\ge$ 0.001 if there are hand-marked ballots. Larger values increase the initial sample size, but make it more likely that the audit will conclude in a single round if the audit finds errors
    + `o2_rate`: expected rate of 2-vote overstatements. Recommended value 0.
    + `u1_rate`: expected rate of 1-vote understatements. Recommended value 0.
    + `u2_rate`: expected rate of 2-vote understatements. Recommended value 0.

* `contests`: a dict of contest-specific data 
    + the keys are unique contest identifiers for contests under audit
    + the values are dicts with keys:
        - `risk_limit`: the risk limit for the audit of this contest
        - `ballots_cast`: an upper bound on the number of cast ballots that contain the contest
        - `choice_function`: `plurality`, `supermajority`, or `IRV`
        - `n_winners`: number of winners for majority contests. (Multi-winner IRV not supported; multi-winner super-majority is nonsense)
        - `share_to_win`: for super-majority contests, the fraction of valid votes required to win, e.g., 2/3.
        - `candidates`: list of names or identifiers of candidates
        - `reported_winners` : list of identifier(s) of candidate(s) reported to have won. Length should equal `n_winners`.
        - `assertion_file`: filename for a set of json descriptors of Assertions (see technical documentation) that collectively imply the reported outcome of the contest is correct. Required for IRV; ignored for other social choice functions

In [2]:
seed = 12345678901234567890  # use, e.g., 20 rolls of a 10-sided die. Seed doesn't have to be numeric
replacement = True  # Sampling without replacement isn't implemented
risk_function = "kaplan_martingale"
g=0.1
N_cards = 300000

In [3]:
cvr_file = './Data/SFDA_2019_Nov8Partial.raire'
manifest_file = './Data/ballot manifest-Sample.xlsx'
manifest_type = 'STYLE'
sample_file = './Data/sample.csv'
mvr_file = './Data/mvr.json'
log_file = './Data/log.json'

In [4]:
error_rates = {'o1_rate':0.002,      # expect 2 1-vote overstatements per 1000 ballots in the CVR stratum
               'o2_rate':0,          # expect 0 2-vote overstatements
               'u1_rate':0,          # expect 0 1-vote understatements
               'u2_rate':0}          # expect 0 2-vote understatements

In [5]:
# contests to audit. Contest 339 is the DA race
contests = {'339':{'risk_limit':0.05,
                     'choice_function':'IRV',
                     'n_winners':1,
                     'candidates':['15','16','17','18','45'],
                     'reported_winners' : ['15'],
                     'assertion_file' : './Data/SF2019Nov8Assertions.json'
                    }
           }

Example of other social choice functions:

> contests =  {'city_council':{'risk_limit':0.05,
                     'choice_function':'plurality',
                     'n_winners':3,
                     'candidates':['Doug','Emily','Frank','Gail','Harry'],
                     'reported_winners' : ['Doug', 'Emily', 'Frank']
                    },
            'measure_1':{'risk_limit':0.05,
                     'choice_function':'supermajority',
                     'share_to_win':2/3,
                     'n_winners':1,
                     'candidates':['yes','no'],
                     'reported_winners' : ['yes']
                    }                  
           }

## Find audit parameters and conduct audit

* For each contest:
    - find claimed outcome by applying SCF to CVRs
    - complain if claimed outcome disagrees with reported outcome
    - construct assertions that imply contest outcome is correct
    - for each assertion:
        + find generalized diluted margin
        
* Find initial (incremental) sample size from smallest diluted margin, for the sampling plan
    - Complain if expected error rates imply any assertion is incorrect

* For each assertion:
    - Initialize discrepancy counts to zero (o1, o2, u1, u2)
    - Initialize measured risk to 1
* While measured risk for any assertion exceeds its risk limit:
    - expand sample by estimated increment
        + identify ballots in manifest
        + update the log file with incremental sample
    - import audit results when ballots have been audited
    - for each assertion:
        + for each sampled ballot:
            - increment discrepancy count for the assertion
        + find measured risk
    - update log file with new measured risks
    - if any measured risk exceeds its risk limit:
        + estimate incremental sample required to complete the audit

In [6]:
# read the assertions for the IRV contest
for c in contests:
    if contests[c]['choice_function'] == 'IRV':
        with open(contests[c]['assertion_file'], 'r') as f:
            contests[c]['assertion_json'] = json.load(f)['audits'][0]['assertions']

In [7]:
# construct the dict of dicts of assertions for each contest
all_assertions = Assertion.make_all_assertions(contests)

In [8]:
all_assertions

{'339': {'18 v 17 elim 15 16 45': <assertion_audit_utils.Assertion at 0x116579710>,
  '17 v 16 elim 15 18 45': <assertion_audit_utils.Assertion at 0x116579550>,
  '15 v 18 elim 16 17 45': <assertion_audit_utils.Assertion at 0x1165796a0>,
  '18 v 16 elim 15 17 45': <assertion_audit_utils.Assertion at 0x116579630>,
  '17 v 16 elim 15 45': <assertion_audit_utils.Assertion at 0x1165795f8>,
  '15 v 17 elim 16 45': <assertion_audit_utils.Assertion at 0x116579470>,
  '15 v 17 elim 16 18 45': <assertion_audit_utils.Assertion at 0x116579390>,
  '18 v 16 elim 15 45': <assertion_audit_utils.Assertion at 0x116579438>,
  '15 v 16 elim 17 45': <assertion_audit_utils.Assertion at 0x1165792e8>,
  '15 v 16 elim 17 18 45': <assertion_audit_utils.Assertion at 0x116579240>,
  '15 v 16 elim 18 45': <assertion_audit_utils.Assertion at 0x116579860>,
  '15 v 16 elim 45': <assertion_audit_utils.Assertion at 0x116579cc0>,
  '15 v 45': <assertion_audit_utils.Assertion at 0x116579c50>}}

## Read the ballot manifest

In [ ]:
# special for SF/Dominion manifest format
manifest = pd.read_excel(manifest_file)

## Read the CVRs 

In [10]:
cvr_input = []
with open(cvr_file) as f:
    cvr_reader = csv.reader(f, delimiter=',', quotechar='"')
    for row in cvr_reader:
        cvr_input.append(row)

print("Read {} rows".format(len(cvr_input)))

Read 118151 rows


In [11]:
# Import CVRs
cvr_list = CVR.from_raire(cvr_input)
print("After merging, there are CVRs for {} cards".format(len(cvr_list)))

After merging, there are CVRs for 118149 ballots


In [16]:
for i in range(10):
    print(cvr_list[i].id)

99813_1_1
99813_1_3
99813_1_6
99813_1_8
99813_1_9
99813_1_11
99813_1_13
99813_1_16
99813_1_17
99813_1_19


In [9]:
# [TO DO!]
# Need to identify which cards in the manifest have CVRs that purport to contain the contest

# Check that each CVR id corresponds to a card in the manifest. If not, complain
# 
n_cvrs = len(cvrs)
manifest, manifest_cards, phantom_cards = prep_dominion_manifest(manifest, N_cards, n_cvrs)

manifest

/Users/stark/Dropbox/Letters/Ms/Vote/Assertion19/Code/dominion_tools.py:28: UserWarning: Manifest does not account for every ballot cast; adding a phantom batch
  warnings.warn('Manifest does not account for every ballot cast; adding a phantom batch')


,Tray #,Tabulator Number,Batch Number,Total Ballots,VBMCart.Cart number,cum_ballots
0,1,99808,81,76,3,76
1,1,99808,78,116,3,192
2,1,99808,80,116,3,308
3,1,99808,79,120,3,428
4,1,99808,77,115,3,543
5,2,99804,9,100,2,643
6,2,99804,6,118,2,761
7,2,99804,7,129,2,890
8,2,99804,8,104,2,994
9,2,99804,10,97,2,1091


In [12]:
# Create CVRs for phantom cards
for i in range(phantom_cards):
    cvr_list.append(CVR(id='phantom_1_'+str(i+1), votes={}))  # matches expected RAIRE id for parsing later
    

len(cvr_list)

415960

In [13]:
manifest_cards

2189

In [ ]:
# find the mean of the assorters for the CVRs and check whether the assertions are met
assorter_means = {}
min_mean = np.infty
for c in contests.keys():
    contests[c]['cvr_means'] = {}
    for asrtn in all_assertions[c]:
        # find mean of the assertion for the CVRs
        amean = all_assertions[c][asrtn].assorter_mean(cvr_list)
        if amean < 1/2:
            warn("assertion {} not satisfied by CVRs: mean value is {}".format(asrtn, amean))
        contests[c]['cvr_means'][asrtn] = amean
        min_mean = np.min([min_mean, amean])

print("minimum assorter mean {}".format(min_mean))

In [ ]:
contests

In [ ]:
check_audit_parameters(risk_function, g, error_rates, contests)

In [ ]:
write_audit_parameters(log_file, seed, replacement, risk_function, g, N_cards,\
                       manifest_cards, phantom_cards, error_rates, contests)

## Set up for sampling

## Estimate initial sample size

In [ ]:
# find initial sample size
initial_size = 100 # FIX ME

## Draw the first sample

In [ ]:
# draw the initial sample
prng = SHA256(seed)
sample = sample_by_index(N_cards, initial_size, prng=prng) # 1-indexed
phantoms = np.sum(sample > manifest_ballots)
print("The sample includes {} phantom ballots, which will be treated conservatively.".format(phantoms))

In [ ]:
# look up the sampled ballots in the manifest
sample_ballots = sample_from_cvrs(cvr_list, sample)

# write the sample
write_ballots_sampled(sample_file, sample_ballots, print_phantoms=False)

## Read the audited sample data

In [ ]:
with open(mvr_file) as f:
    mvr_json = json.load(f)

mvr = CVR.from_dict(mvr_json['ballots'])

In [ ]:
for i in range(5):
    print(mvr[i].id, mvr[i].votes)

## Find measured risks for all assertions

### Deal with Phantoms/Zombies

Given an independent (i.e., not relying on the voting system) upper bound on the number of cards that contain the contest, if the number of CVRs that contain the contest does not exceed that bound, we can sample from paper purported to contain the contest and use the "zombies" approach (Banuelos & Stark) to deal with missing CVRs. This can greatly increase the efficiency of the audit if the contest is on only a small percentage of the ballots.

Any sampled phantom card (i.e., a card for which there are no CVRs) is treated as if its CVR is a non-vote (which it is), and as if its MVR was least favorable (a "zombie" producing the greatest doubt in every assertion, separately). Any sampled card for which
there is a CVR is compared to its corresponding CVR. 
If the card turns out not to contain the contest (despite the fact that the CVR says it does), the MVR is treated in the least favorable way for each assertion (i.e., as a zombie rather than as a non-vote).

In [ ]:
# adjust for phantoms

In [ ]:
# Identify assertions not yet confirmed

In [ ]:
# Log the status of the audit 

# Escalation: how many more ballots should be drawn?

This tool estimates how many more ballots will need to be audited to confirm any remaining contests. The enlarged sample size is based on:

* ballots already sampled
* assumption that we will continue to see overstatements and understatements at the same rate that observed in the sample

In [ ]:
sample_sizes_new = {}

# TBD


In [ ]:
# augment the sample
# reset the seed
prng = SHA256(seed)
old_sample = sample
sample = sample_by_index(N_cards, sample_size, prng=prng)
incremental_sample = np.sort(list(set(sample) - set(old_sample)))
